In [28]:
import pandas as pd
import transformers
from transformers import pipeline
import numpy as np
from tqdm import tqdm

In [3]:
books = pd.read_csv('../dataset/books_clean.csv')
books.head()

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."


In [6]:
cat = books['categories'].value_counts().reset_index()
cat

,categories,count
0,Fiction,2111
1,Juvenile Fiction,390
2,Biography & Autobiography,311
3,History,207
4,Literary Criticism,124
...,...,...
474,Conspiracies,1
475,Brothers and sisters,1
476,Rock musicians,1
477,Community life,1


In [8]:
cat.query('count > 50')

,categories,count
0,Fiction,2111
1,Juvenile Fiction,390
2,Biography & Autobiography,311
3,History,207
4,Literary Criticism,124
5,Philosophy,117
6,Religion,117
7,Comics & Graphic Novels,116
8,Drama,86
9,Juvenile Nonfiction,57


In [11]:
category_mapping = {
    'Fiction' : 'Fiction',
    'Juvenile Fiction' : "children's Fiction",
    'Biography & Autobiography':"Nonfiction",
    'History':"Nonfiction",
    'Literary Criticism':"Nonfiction",
    'Philosophy':"Nonfiction",
    'Religion':"Nonfiction",
    'Comics & Graphic Novels':'Fiction',
    'Drama':'Fiction',
    'Juvenile Nonfiction':"children's Nonfiction",
    'Science':"Nonfiction",
    'Poetry':'Fiction'
}

In [ ]:
#books['simple_categories'] = books['categories'].map(category_mapping)

Zero-shot classification

In [18]:
pipe = pipeline("zero-shot-classification", model='facebook/bart-large-mnli')

c:\Users\deep1\OneDrive\Documents\Book_Rec_LLM_Project\book_rec\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\deep1\.cache\huggingface\hub\models--facebook--bart-large-mnli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Device set to use cpu


In [20]:
category = ['Fiction', 'Juvenile Fiction','Biography & Autobiography','History', 'Literary Criticism', 'Philosophy', 'Religion','Comics & Graphic Novels','Drama', 'Juvenile Nonfiction','Science', 'Poetry']

In [21]:
sequence = books.loc[books['categories'] == 'Fiction','description'].reset_index(drop=True)[0]

In [22]:
sequence

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [23]:
pipe(sequence,category)

{'sequence': 'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst

In [26]:
max_index = np.argmax(pipe(sequence,category)['scores'])
max_label = pipe(sequence,category)['labels'][max_index]

In [27]:
max_label

'Fiction'

In [53]:
def generate_predcition(sequence,categories):
    predictions = pipe(sequence,categories)
    max_index = np.argmax(predictions['scores'])
    max_label = predictions['labels'][max_index]
    return max_label    

In [56]:
actual_Cats = []
predicated_cats = []

for i in tqdm(range(0,300)):
    sequence = books.loc[books['categories']=='Fiction','description'].reset_index(drop=True)[i]
    predicated_cats += [generate_predcition(sequence,category)]
    actual_Cats += ['Fiction']

100%|██████████| 300/300 [1:11:27<00:00, 14.29s/it]


In [64]:
for i in tqdm(range(0,100)):
    sequence = books.loc[books['categories']=='History','description'].reset_index(drop=True)[i]
    predicated_cats += [generate_predcition(sequence,category)]
    actual_Cats += ['History']

100%|██████████| 100/100 [25:31<00:00, 15.32s/it]


In [65]:
for i in tqdm(range(0,80)):
    sequence = books.loc[books['categories']=='Literary Criticism','description'].reset_index(drop=True)[i]
    predicated_cats += [generate_predcition(sequence,category)]
    actual_Cats += ['Literary Criticism']

  0%|          | 0/80 [00:00<?, ?it/s]

100%|██████████| 80/80 [21:43<00:00, 16.29s/it]


In [66]:
for i in tqdm(range(0,100)):
    sequence = books.loc[books['categories']=='Biography & Autobiography','description'].reset_index(drop=True)[i]
    predicated_cats += [generate_predcition(sequence,category)]
    actual_Cats += ['Biography & Autobiography']

100%|██████████| 100/100 [25:11<00:00, 15.12s/it]


In [67]:
for i in tqdm(range(0,50)):
    sequence = books.loc[books['categories']=='Drama','description'].reset_index(drop=True)[i]
    predicated_cats += [generate_predcition(sequence,category)]
    actual_Cats += ['Drama']

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [11:50<00:00, 14.21s/it]


In [68]:
for i in tqdm(range(0,70)):
    sequence = books.loc[books['categories']=='Philosophy','description'].reset_index(drop=True)[i]
    predicated_cats += [generate_predcition(sequence,category)]
    actual_Cats += ['Philosophy']

100%|██████████| 70/70 [20:34<00:00, 17.64s/it]


In [69]:
for i in tqdm(range(0,50)):
    sequence = books.loc[books['categories']=='Religion','description'].reset_index(drop=True)[i]
    predicated_cats += [generate_predcition(sequence,category)]
    actual_Cats += ['Religion']

100%|██████████| 50/50 [3:02:59<00:00, 219.60s/it]     


In [70]:
for i in tqdm(range(0,50)):
    sequence = books.loc[books['categories']=='Science','description'].reset_index(drop=True)[i]
    predicated_cats += [generate_predcition(sequence,category)]
    actual_Cats += ['Science']

100%|██████████| 50/50 [12:19<00:00, 14.79s/it]


In [71]:
for i in tqdm(range(0,50)):
    sequence = books.loc[books['categories']=='Poetry','description'].reset_index(drop=True)[i]
    predicated_cats += [generate_predcition(sequence,category)]
    actual_Cats += ['Poetry']

100%|██████████| 50/50 [11:34<00:00, 13.88s/it]


In [72]:
for i in tqdm(range(0,150)):
    sequence = books.loc[books['categories']=='Juvenile Fiction','description'].reset_index(drop=True)[i]
    predicated_cats += [generate_predcition(sequence,category)]
    actual_Cats += ['Juvenile Fiction']

  0%|          | 0/150 [00:00<?, ?it/s]

100%|██████████| 150/150 [35:11<00:00, 14.08s/it] 


In [73]:
for i in tqdm(range(0,50)):
    sequence = books.loc[books['categories']=='Juvenile Nonfiction','description'].reset_index(drop=True)[i]
    predicated_cats += [generate_predcition(sequence,category)]
    actual_Cats += ['Juvenile Nonfiction']

100%|██████████| 50/50 [06:21<00:00,  7.64s/it]


In [74]:
for i in tqdm(range(0,50)):
    sequence = books.loc[books['categories']=='Comics & Graphic Novels','description'].reset_index(drop=True)[i]
    predicated_cats += [generate_predcition(sequence,category)]
    actual_Cats += ['Comics & Graphic Novels']

100%|██████████| 50/50 [08:28<00:00, 10.17s/it]


In [75]:
predicated_df = pd.DataFrame({'actual_categories':actual_Cats, "predicted_categories":predicated_cats})
predicated_df.to_csv('../dataset/predicated_df.csv')
predicated_df

,actual_categories,predicted_categories
0,Fiction,Fiction
1,Fiction,Drama
2,Fiction,Drama
3,Fiction,Drama
4,Fiction,Drama
...,...,...
1095,Comics & Graphic Novels,Fiction
1096,Comics & Graphic Novels,Biography & Autobiography
1097,Comics & Graphic Novels,Biography & Autobiography
1098,Comics & Graphic Novels,Fiction


In [76]:
predicated_df['correct_prediction'] = (
    np.where(predicated_df['actual_categories']==predicated_df['predicted_categories'],1,0)
)

In [77]:
predicated_df

,actual_categories,predicted_categories,correct_prediction
0,Fiction,Fiction,1
1,Fiction,Drama,0
2,Fiction,Drama,0
3,Fiction,Drama,0
4,Fiction,Drama,0
...,...,...,...
1095,Comics & Graphic Novels,Fiction,0
1096,Comics & Graphic Novels,Biography & Autobiography,0
1097,Comics & Graphic Novels,Biography & Autobiography,0
1098,Comics & Graphic Novels,Fiction,0


In [78]:
predicated_df['correct_prediction'].sum() / len(predicated_df)

0.4036363636363636